Bugün ödev kapsamında youtube' da bulunan bir kanaldan veri çekeceğiz ve incelemeler yapacağız.

Youtubeda bulunan bir kanalın channel id'sine aşağıdaki linkten ulaşabilirsiniz
- https://www.streamweasels.com/tools/youtube-channel-id-and-user-id-convertor/

TEDx
Channel ID: UCsT0YIqwnpJCM-mx7-gSA4Q
- En başta bu kanalı denedim fakat sürekli yeni videolar attığı için daha stabil bir kanal seçmek istedim. Tedx kanalı için deneme yaptım burada da son 50 veriye erişim izni verildi.

TED
CHANNEL_ID ="UCAuUUnT6oDeKwE6v1NGQxug"
- Ted kanalının istatistiklerini çekmek istediğimde hata aldım. Araştırdığımda API anahtarımın ted kanalının verilerine erişim izni olmadığı sonucuna vardım.


1MILLION_Dance
Channel ID: UCw8ZhLPdQ0u_Y-TLKd61hGA


In [1]:
import requests
import pandas as pd
import time
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Videolarini cekmek istedigin kanalin id sini kullanıyoruz

CHANNEL_ID ="UCw8ZhLPdQ0u_Y-TLKd61hGA"  #1MillionDance channel id
pageToken = ""   #Bos string olarak tanimlıyoruz bu asağıdaki koddan sonra doluyor

url = "https://www.googleapis.com/youtube/v3/search?key="+YOUTUBE_API_KEY+"&channelId="+CHANNEL_ID+"&part=snippet,id&order=date&maxResults=1000"+pageToken

In [ ]:
response= requests.get(url).json()

In [ ]:
response

response["items"][0]

In [ ]:
response["items"][0]['id']['videoId']

In [ ]:
response["items"][0]['snippet']['title']

In [ ]:
response["items"][0]['snippet']['publishedAt']

In [ ]:
upload_time = response["items"][0]['snippet']['publishedAt']
upload_time = str(upload_time).split("T")[0]
#Ustte tarihin nasil oldugunu gorduk T den ayirip daha hos bir goruntu elde ediyoruz
upload_time

In [ ]:
# for dongusu ile video baslıklarını cekiyoruz
#for loop in response
for video in response["items"]:
  if video["id"]["kind"] == "youtube#video":
    video_title = video["snippet"]["title"]
    print(video_title)

In [ ]:
video_id = "iOBknW-K_ZA"
video_url_endpoint = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+YOUTUBE_API_KEY

video_response = requests.get(video_url_endpoint).json()

video_response

In [ ]:
# Görüntülenme, beğenme sayısı, favori sayısı, yorum sayısı bilgilerini verir
video_response["items"][0]["statistics"]

In [ ]:
def get_video_statistics(video_id, youtube_api_key):
    # collecting video statistics
    video_stats_url = "https://www.googleapis.com/youtube/v3/videos?id="+video_id+"&part=statistics&key="+youtube_api_key
    video_stats_response = requests.get(video_stats_url).json()

    view_count = video_stats_response['items'][0]["statistics"]["viewCount"]
    like_count = video_stats_response['items'][0]["statistics"]["likeCount"]
    fav_count = video_stats_response['items'][0]["statistics"]["favoriteCount"]
    comment_count = video_stats_response['items'][0]["statistics"]["commentCount"]


    return view_count, like_count, fav_count, comment_count


In [ ]:
def get_videos_from_channel(df, channel_id, youtube_api_key):
    # make API call
    pageToken = ""
    url = "https://www.googleapis.com/youtube/v3/search?key="+youtube_api_key+"&channelId="+channel_id+"&part=snippet,id&order=date&maxResults=1000"+pageToken

    response = requests.get(url).json()
    time.sleep(3)

    for video in response['items']:
        if video['id']['kind'] == 'youtube#video':
            video_title = video['snippet']['title']
            video_id = video['id']['videoId']
            upload_date = video['snippet']['publishedAt']
            upload_date= str(upload_date).split("T")[0]

            view_count, like_count, fav_count, comment_count = get_video_statistics(video_id, youtube_api_key)

            #save in pandas df
            df = df._append({
                "video_title": video_title,
                "video_id": video_id,
                "upload_date": upload_date,
                "view_count": view_count,
                "like_count": like_count,
                "favorite_count": fav_count,
                "comment_count": comment_count
            },
            ignore_index = True
            )
    return df

In [ ]:
df = pd.DataFrame(columns=["video_title","video_id","upload_date","view_count","like_count", "favorite_count", "comment_count"])
choreography_data = get_videos_from_channel(df, CHANNEL_ID, YOUTUBE_API_KEY)

In [ ]:
choreography_data.info()

In [ ]:
choreography_data.head()

In [ ]:
choreography_data['view_count'] = choreography_data['view_count'].astype(int)
choreography_data['like_count'] = choreography_data['like_count'].astype(int)
choreography_data['favorite_count'] = choreography_data['favorite_count'].astype(int)
choreography_data['comment_count'] = choreography_data['comment_count'].astype(int)


In [ ]:
choreography_data['upload_date'] = pd.to_datetime(choreography_data['upload_date'], format='%Y-%m-%d')

In [ ]:
choreography_data.describe()

In [ ]:
choreography_data.size;
#sonuna noktalı virgül koyarsak kodu çalıştırır fakat bize çıktıyı göstermez
#Bunu az önce yanlışlıkla keşfettiğim için not olarak eklemek istedim

In [ ]:
choreography_data.shape

In [ ]:
choreography_data.size

In [ ]:
bar1 = sns.barplot(x="video_title",  y="view_count", data=choreography_data, color='darkblue')

In [ ]:
#en fazla görüntüleme alan videoyu bulmak icin
max_view = choreography_data['view_count'].max()
max_view_video = choreography_data.loc[choreography_data['view_count'] == max_view, 'video_title'].iloc[0]
print("En fazla görüntülenen video:", max_view_video, max_view)


In [ ]:
#en cok görüntülenen videoyu bulmak icin farklı bir yol
# 'view_count' sütunundaki maksimum değerin indeksini bulalım
max_index = choreography_data['view_count'].idxmax()

# Maksimum değere karşılık gelen diğer sütunları yazdıralım
max_view_video = choreography_data.loc[max_index, 'video_title']
max_view_count = choreography_data.loc[max_index, 'view_count']

print(f"En çok görüntülenen video: '{max_view_video}' adında {max_view_count}")

In [ ]:
#en az izlenen video için
min_index = choreography_data['view_count'].idxmin()

# Maksimum değere karşılık gelen diğer sütunları yazdıralım
min_view_video = choreography_data.loc[min_index, 'video_title']
min_view_count = choreography_data.loc[min_index, 'view_count']

print(f"En az görüntülenen video: '{min_view_video}' adında {min_view_count}")